In [1]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player2_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to Player 2.
    """
    visited = set()
    queue = deque(player2_cells)
    for cell in player2_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 9,
    m: int = 3,
    n: int = 3
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_unreachable_cells)
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Unreachable cells are empty cells not in 'reachable'
                # Exclude the cell just filled by Player 1
                remaining_empty = all_empty - {cell}
                unreachable = remaining_empty - reachable
                
                if unreachable:
                    # Found a move that cordons off some cells
                    cordoned_moves.append((cell, sorted(list(unreachable))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    return cordoned_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nCompetitive Sudoku Grid:")
    print("\n    " + "  ".join(str(i) for i in range(len(grid))))
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nGrid with Unreachable Cells Highlighted:")
    print("\n    " + "  ".join(str(i) for i in range(len(grid))))
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append("X")
            else:
                row_display.append(legend.get(cell, str(cell)))
        print(f"{r}  " + "  ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup
    # 0 = empty, 1 = Player 1, 2 = Player 2
    player1 = [(1, 1), (1, 0), (2, 1), (3, 1), (4, 1)]
    player2 = [(7, 8), (8, 7), (8, 8)]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(9)] for _ in range(9)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to make the following cells unreachable to Player 2:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")


Initial Grid Setup:

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8
0  .  .  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  .  .  .  .  .  .  .
6  .  .  .  .  .  .  .  .  .
7  .  .  .  .  .  .  .  .  P2
8  .  .  .  .  .  .  .  P2  P2

Player 1 can fill cell (0, 1) to make the following cells unreachable to Player 2:
[(0, 0)]

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8
0  .  P1  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  .  .  .  .  .  .  .
6  .  .  .  .  .  .  .  .  .
7  .  .  .  .  .  .  .  .  P2
8  .  .  .  .  .  .  .  P2  P2


Grid with Unreachable Cells Highlighted:

    0  1  2  3  4  5  6  7  8
0  X  P1  .  .  .  .  .  .  .
1  P1  P1  .  .  .  .  .  .  .
2  .  P1  .  .  .  .  .  .  .
3  .  P1  .  .  .  .  .  .  .
4  .  P1  .  .  .  .  .  .  .
5  .  .  

In [2]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player2_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to Player 2.
    """
    visited = set()
    queue = deque(player2_cells)
    for cell in player2_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 9,
    m: int = 3,
    n: int = 3
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_unreachable_cells)
    Only returns moves that block the maximum number of cells.
    If multiple moves block the same maximum number, all are returned.
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Unreachable cells are empty cells not in 'reachable'
                # Exclude the cell just filled by Player 1
                remaining_empty = all_empty - {cell}
                unreachable = remaining_empty - reachable
                
                if unreachable:
                    # Found a move that cordons off some cells
                    cordoned_moves.append((cell, sorted(list(unreachable))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    if not cordoned_moves:
        return []
    
    # Determine the maximum number of blocked cells
    max_blocked = max(len(unreachable) for _, unreachable in cordoned_moves)
    
    # Filter moves that have the maximum number of blocked cells
    best_moves = [
        (cell, unreachable)
        for cell, unreachable in cordoned_moves
        if len(unreachable) == max_blocked
    ]
    
    return best_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nCompetitive Sudoku Grid:\n")
    print("    " + "  ".join(str(i) for i in range(len(grid))))
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    print("\nGrid with Unreachable Cells Highlighted:\n")
    print("    " + "  ".join(str(i) for i in range(len(grid))))
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append("X")
            else:
                row_display.append(legend.get(cell, str(cell)))
        print(f"{r}  " + "  ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup
    # 0 = empty, 1 = Player 1, 2 = Player 2
#     player1 = [(1, 1), (2, 1), (2, 0), (3, 1), (4, 1), (5, 1)]
#     player2 = [(7, 8), (8, 7), (8, 8)]
    
#     # Initialize grid for visualization
#     initial_grid = [[0 for _ in range(9)] for _ in range(9)]
#     for (r, c) in player1:
#         initial_grid[r][c] = 1
#     for (r, c) in player2:
#         initial_grid[r][c] = 2


    # Player 1 occupies cells forming a partial barrier around an internal area
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 # (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        # Determine the maximum number of blocked cells
        max_blocked = len(cordoned_scenarios[0][1])
        print(f"Maximum number of cells blocked: {max_blocked}\n")
        
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to block {len(unreachable)} cells:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")


Initial Grid Setup:

Competitive Sudoku Grid:

    0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
3  .  .  P2  .  .  .  .  .  .  .  .  .  .  .  .  .
4  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
5  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
6  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
7  .  .  .  .  .  P1  .  .  .  .  .  .  .  .  .  .
8  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
9  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
11  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
12  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
13  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
14  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
15  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

Maximum number of cells blocked: 5

Player 1 can fill cell (6, 4) to block

In [3]:
from typing import List, Tuple, Set
from collections import deque
import copy

# Define type aliases for clarity
Cell = Tuple[int, int]
Grid = List[List[int]]

def get_block_indices(row: int, col: int, m: int, n: int) -> Tuple[int, int]:
    """
    Return the top-left corner of the block containing (row, col).
    """
    return (row - row % m, col - col % n)

def is_valid_move(grid: Grid, row: int, col: int, number: int, m: int, n: int) -> bool:
    """
    Check if placing 'number' at (row, col) is valid according to Sudoku rules.
    """
    N = m * n
    # Check row
    if number in grid[row]:
        return False
    # Check column
    for r in range(N):
        if grid[r][col] == number:
            return False
    # Check block
    start_row, start_col = get_block_indices(row, col, m, n)
    for r in range(start_row, start_row + m):
        for c in range(start_col, start_col + n):
            if grid[r][c] == number:
                return False
    return True

def get_adjacent_cells(row: int, col: int, N: int) -> List[Cell]:
    """
    Return all adjacent cells (including diagonals) for a given cell.
    """
    directions = [(-1, -1), (-1, 0), (-1, 1),
                  (0, -1),          (0, 1),
                  (1, -1),  (1, 0), (1, 1)]
    adj = []
    for dr, dc in directions:
        r, c = row + dr, col + dc
        if 0 <= r < N and 0 <= c < N:
            adj.append((r, c))
    return adj

def get_reachable_cells(grid: Grid, player_cells: List[Cell], N: int) -> Set[Cell]:
    """
    Perform BFS to find all cells reachable to the given player.
    """
    visited = set()
    queue = deque(player_cells)
    for cell in player_cells:
        visited.add(cell)
    
    while queue:
        current = queue.popleft()
        for adj in get_adjacent_cells(current[0], current[1], N):
            if adj not in visited:
                cell_value = grid[adj[0]][adj[1]]
                if cell_value == 0 or cell_value == 2:
                    visited.add(adj)
                    queue.append(adj)
    return visited

def find_cordoned_off_move(
    player1_cells: List[Cell],
    player2_cells: List[Cell],
    N: int = 16,
    m: int = 4,
    n: int = 4
) -> List[Tuple[Cell, List[Cell]]]:
    """
    Identify cells where Player 1 can fill to cordon off areas from Player 2.
    
    Returns a list of tuples:
        (cell_to_fill, list_of_newly_unreachable_cells)
    Only returns moves that block the maximum number of additional cells.
    If multiple moves block the same maximum number, all are returned.
    If no moves can block any cells, returns an empty list.
    """
    # Initialize grid: 0 = empty, 1 = Player 1, 2 = Player 2
    grid = [[0 for _ in range(N)] for _ in range(N)]
    for (r, c) in player1_cells:
        grid[r][c] = 1
    for (r, c) in player2_cells:
        grid[r][c] = 2
    
    # Initial reachable cells for Player 2
    initial_reachable = get_reachable_cells(grid, player2_cells, N)
    
    # Find all empty cells adjacent to Player 1's cells
    adjacent_empty_cells = set()
    for (r, c) in player1_cells:
        for adj in get_adjacent_cells(r, c, N):
            if grid[adj[0]][adj[1]] == 0:
                adjacent_empty_cells.add(adj)
    
    # If no adjacent empty cells, no possible moves to block
    if not adjacent_empty_cells:
        return []
    
    # Find all empty cells (for unreachable determination later)
    all_empty = set()
    for r in range(N):
        for c in range(N):
            if grid[r][c] == 0:
                all_empty.add((r, c))
    
    cordoned_moves = []
    
    for cell in adjacent_empty_cells:
        r, c = cell
        # Try all possible numbers for this cell
        for number in range(1, N + 1):
            if is_valid_move(grid, r, c, number, m, n):
                # Simulate filling the cell with Player 1's number
                simulated_grid = copy.deepcopy(grid)
                simulated_grid[r][c] = 1  # Player 1 fills the cell
                
                # Determine reachable cells for Player 2 after the move
                new_reachable = get_reachable_cells(simulated_grid, player2_cells, N)
                
                # Newly blocked cells are those that were reachable before but not after
                newly_blocked = initial_reachable - new_reachable
                
                if newly_blocked:
                    # Found a move that blocks some cells
                    cordoned_moves.append((cell, sorted(list(newly_blocked))))
                    # Assuming one number is enough to cordon off for this cell
                    break  # Proceed to next cell after finding one valid number
    
    if not cordoned_moves:
        return []
    
    # Determine the maximum number of newly blocked cells
    max_blocked = max(len(unreachable) for _, unreachable in cordoned_moves)
    
    # Filter moves that have the maximum number of blocked cells
    best_moves = [
        (cell, unreachable)
        for cell, unreachable in cordoned_moves
        if len(unreachable) == max_blocked
    ]
    
    return best_moves

def print_grid(grid: Grid):
    """
    Print the grid in a readable format.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    # Header with column numbers
    header = "     " + "  ".join(f"{i:2}" for i in range(len(grid)))
    print("\nCompetitive Sudoku Grid:\n")
    print(header)
    for idx, row in enumerate(grid):
        row_display = "  ".join(legend.get(cell, str(cell)) for cell in row)
        print(f"{idx:2}  {row_display}")
    print()

def visualize_unreachable(grid: Grid, unreachable: List[Cell]):
    """
    Print the grid highlighting unreachable cells with 'X'.
    """
    legend = {
        0: ".",
        1: "P1",
        2: "P2"
    }
    # Header with column numbers
    header = "     " + "  ".join(f"{i:2}" for i in range(len(grid)))
    print("\nGrid with Unreachable Cells Highlighted:\n")
    print(header)
    for r, row in enumerate(grid):
        row_display = []
        for c, cell in enumerate(row):
            if (r, c) in unreachable:
                row_display.append(" X")
            else:
                display = legend.get(cell, str(cell))
                # Adjust spacing for single-digit numbers
                if isinstance(display, int) and display < 10:
                    display = f" {display}"
                row_display.append(display)
        print(f"{r:2}  " + " ".join(row_display))
    print()

# Example Usage
if __name__ == "__main__":
    # Example grid setup for a 16x16 grid
    # 0 = empty, 1 = Player 1, 2 = Player 2
    # We'll create a scenario where Player 1 can block an internal area
    # Additionally, we'll create a test case where the grid is already fully blocked

    # Incorrectly states blocked off scenario, because area is already blocked off    
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2
    
    print("Initial Grid Setup:")
    print_grid(initial_grid)
    
    # Find cordoned-off moves
    cordoned_scenarios = find_cordoned_off_move(player1, player2)
    
    if cordoned_scenarios:
        # Determine the maximum number of blocked cells
        max_blocked = len(cordoned_scenarios[0][1])
        print(f"Maximum number of cells blocked: {max_blocked}\n")
        
        for move, unreachable in cordoned_scenarios:
            r, c = move
            print(f"Player 1 can fill cell ({r}, {c}) to block {len(unreachable)} cells:")
            print(unreachable)
            
            # Simulate the move for visualization
            simulated_grid = copy.deepcopy(initial_grid)
            simulated_grid[r][c] = 1  # Player 1 fills the cell
            
            print_grid(simulated_grid)
            visualize_unreachable(simulated_grid, unreachable)
    else:
        print("No single-cell move found that cordons off any area for Player 2.")

Initial Grid Setup:

Competitive Sudoku Grid:

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
 3  .  .  P2  .  .  .  .  .  .  .  .  .  .  .  .  .
 4  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 5  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
 6  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
 7  .  .  .  .  .  P1  .  .  P1  .  .  .  .  .  .  .
 8  .  .  .  .  .  P1  P1  P1  P1  .  .  .  .  .  .  .
 9  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
10  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
11  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
12  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
13  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
14  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
15  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .

Maximum number of cells blocked: 1

Player 1 can fil

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  . . . . . . . . . . . . . . . .
 1  . . . . . . . . . . . . . . . .
 2  . . P2 P2 P2 . . . . . . . . . . .
 3  . . P2 . . . . . . . . . . . . .
 4  . . . . . . . . . . . . . . . .
 5  . . . . . P1 P1 P1 P1 . . . . . . .
 6  . . . . . P1 . . P1 . . . . . . .
 7  . . . . . P1 . . P1 . . . . . . .
 8  . . . . . P1 P1 P1 P1 . . . . . . .
 9  . . . . . . .  X . . . . . . . .
10  . . . . . . . . . . . . . . . .
11  . . . . . . . . . . . . . . . .
12  . . . . . . . . . . . . . . . .
13  . . . . . . . . . . . . . . . .
14  . . . . . . . . . . . . . . . .
15  . . . . . . . . . . . . . . . .

Player 1 can fill cell (6, 4) to block 1 cells:
[(6, 4)]

Competitive Sudoku Grid:

      0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 0  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 1  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .
 2  .  .  P2  P2  P2  .  .  .  .  .  .  .  .  .  .  .
 3  .  .  P2  .  .  .  .  .  .

In [4]:
# Two equal sized blocked off areas can be created, return both    
    
    player1 = [(1, 1), (1, 0), (2, 1), (3, 1), (4, 1)]
    player2 = [(7, 8), (8, 7), (8, 8)]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(9)] for _ in range(9)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2

IndentationError: unexpected indent (<ipython-input-4-b6a2dd1e8ac8>, line 3)

In [ ]:
# Incorrectly states blocked off scenario, because area is already blocked off    
    player1 = [
        (5, 5), (5, 6), (5, 7), (5, 8),
        (6, 5),                 (6, 8),
        (7, 5),                 (7, 8),
        (8, 5), (8, 6), (8, 7), (8, 8)
    ]
    
    # Player 2 occupies cells outside the barrier
    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2

In [ ]:
# Example not blocked off because opponents cell in the 'wall"

    player1 = [
#         (5, 5), 
                (5, 6), (5, 7), (5, 8),
        (6, 5),           (6, 8),
        (7, 5),           (7, 8),
        (8, 5),        (8, 6), (8, 7), (8, 8)
    ]
    

    player2 = [
        (2, 2), (2, 3), (2, 4),
        (3, 2), (8, 6)
    ]
    
    # Initialize grid for visualization
    initial_grid = [[0 for _ in range(16)] for _ in range(16)]
    for (r, c) in player1:
        initial_grid[r][c] = 1
    for (r, c) in player2:
        initial_grid[r][c] = 2